# Ejercicios Semana 5

In [38]:
import json
import requests
from pprint import pprint

## Problemas de Archivos

In [2]:
!wget "https://github.com/arleserp/MinTIC2022/raw/master/files.zip" -q
!unzip -u -q files.zip

Dado el archivo de texto files/SalesJan2009.csv:

1. Procese el archivo para obtener las compras realizadas en un país dado.
2. Procese el archivo para obtener las compras realizadas con un medio de pago dado.

In [9]:
def read_csv(path):
    data = []
    with open(path, 'r', encoding="utf-8") as f:
        data = f.readlines() # Lista
    rows = [row.split(',') for row in data]
    head = rows.pop(0)
    elementos = []
    for row in rows:
        d = {}
        indice = 0
        while indice < len(head):
          d[head[indice]] = row[indice]
          indice += 1
        for indice in range(len(head)):
          d[atributo] = row[indice]
        #for indice, atributo in enumerate(head):
            
        elementos.append(d)
    return elementos

In [5]:
t = read_csv('files/SalesJan2009.csv')

['Transaction_date,Product,Price,Payment_Type,Name,City,State,Country,Account_Created,Last_Login,Latitude,Longitude\n', '1/2/2009 6:17,Product1,1200,Mastercard,carolina,Basildon,England,United Kingdom,1/2/2009 6:00,1/2/2009 6:08,51.5,-1.1166667\n', '1/2/2009 4:53,Product1,1200,Visa,Betina,Parkville                   ,MO,United States,1/2/2009 4:42,1/2/2009 7:49,39.195,-94.68194\n', '1/2/2009 13:08,Product1,1200,Mastercard,Federica e Andrea,Astoria                     ,OR,United States,1/1/2009 16:21,1/3/2009 12:32,46.18806,-123.83\n', '1/3/2009 14:44,Product1,1200,Visa,Gouya,Echuca,Victoria,Australia,9/25/2005 21:13,1/3/2009 14:22,-36.1333333,144.75\n', '1/4/2009 12:56,Product2,3600,Visa,Gerd W ,Cahaba Heights              ,AL,United States,11/15/2008 15:47,1/4/2009 12:45,33.52056,-86.8025\n', '1/4/2009 13:19,Product1,1200,Visa,LAURENCE,Mickleton                   ,NJ,United States,9/24/2008 15:19,1/4/2009 13:04,39.79,-75.23806\n', '1/4/2009 20:11,Product1,1200,Mastercard,Fleur,Peoria 

In [10]:
t = read_csv('files/SalesJan2009.csv')

['Transaction_date', 'Product', 'Price', 'Payment_Type', 'Name', 'City', 'State', 'Country', 'Account_Created', 'Last_Login', 'Latitude', 'Longitude\n']


In [11]:
t

[{'Account_Created': '1/2/2009 6:00',
  'City': 'Basildon',
  'Country': 'United Kingdom',
  'Last_Login': '1/2/2009 6:08',
  'Latitude': '51.5',
  'Longitude\n': '-1.1166667\n',
  'Name': 'carolina',
  'Payment_Type': 'Mastercard',
  'Price': '1200',
  'Product': 'Product1',
  'State': 'England',
  'Transaction_date': '1/2/2009 6:17'},
 {'Account_Created': '1/2/2009 4:42',
  'City': 'Parkville                   ',
  'Country': 'United States',
  'Last_Login': '1/2/2009 7:49',
  'Latitude': '39.195',
  'Longitude\n': '-94.68194\n',
  'Name': 'Betina',
  'Payment_Type': 'Visa',
  'Price': '1200',
  'Product': 'Product1',
  'State': 'MO',
  'Transaction_date': '1/2/2009 4:53'},
 {'Account_Created': '1/1/2009 16:21',
  'City': 'Astoria                     ',
  'Country': 'United States',
  'Last_Login': '1/3/2009 12:32',
  'Latitude': '46.18806',
  'Longitude\n': '-123.83\n',
  'Name': 'Federica e Andrea',
  'Payment_Type': 'Mastercard',
  'Price': '1200',
  'Product': 'Product1',
  'Stat

In [12]:
total = 0
for row in t:
  if row['Country'] == 'France':
    total += float(row['Price'])
total

53100.0

In [13]:
total = 0
for row in t:
  if row['Payment_Type'] == 'Mastercard':
    total += float(row['Price'])
total

458450.0

In [14]:
def extract_values(value, match, attribute, items):
    find = []
    for i in items:
        if value == i[match]:
            find.append(i[attribute])
    return find

In [25]:
filtered = extract_values('France', 'Country', 'Price', read_csv('files/SalesJan2009.csv'))
filtered[:10]

['Transaction_date', 'Product', 'Price', 'Payment_Type', 'Name', 'City', 'State', 'Country', 'Account_Created', 'Last_Login', 'Latitude', 'Longitude\n']


['1200',
 '1200',
 '3600',
 '7500',
 '1200',
 '1200',
 '3600',
 '1200',
 '1200',
 '1200']

In [19]:
sum([float(p) for p in filtered])

53100.0

In [20]:
filtered = extract_values('Mastercard', 'Payment_Type', 'Price', read_csv('files/SalesJan2009.csv'))
filtered[:2]

['Transaction_date', 'Product', 'Price', 'Payment_Type', 'Name', 'City', 'State', 'Country', 'Account_Created', 'Last_Login', 'Latitude', 'Longitude\n']


['1200', '1200']

In [21]:
sum([float(p) for p in filtered])

458450.0

## Problemas JSON

Usando el API [JSONPlaceholder](https://jsonplaceholder.typicode.com/guide/), realizar las siguientes operaciones

* [x] Consultar los usuarios y crear una lista en donde se muestre el Identificador y el nombre como una cadena de caracteres.
* [x]  Consultar todos los posts y crear una lista de diccionarios en donde solo se presente el identificador y el titulo.
* [x]  Capturar un identificador ingresado por el usuario y verificar que exista en la lista de usuarios disponibles consultada en el punto 1 y mostrar su información relevante.
* [x]  Con el identificador capturado en el punto 3 identificar cuales posts de los consultados en el punto dos le pertenecen al usuario y crear un nuevo diccionario en donde este la información del usuario y un listado de sus posts.
* [x]  Guardar el resultado del punto 4 en un archivo JSON
* [ ] Capturar un identificador ingresado por el usuario y verificar que exista en la lista de post que se consulto en el punto 2 y consultar los comentarios de este post 

In [43]:
def consultar_usuarios():
    response = requests.get('https://jsonplaceholder.typicode.com/users')
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception('No se pudieron consultar los usuarios')
        return []

In [36]:
consultar_usuarios()[:1]

[{'address': {'city': 'Gwenborough',
   'geo': {'lat': '-37.3159', 'lng': '81.1496'},
   'street': 'Kulas Light',
   'suite': 'Apt. 556',
   'zipcode': '92998-3874'},
  'company': {'bs': 'harness real-time e-markets',
   'catchPhrase': 'Multi-layered client-server neural-net',
   'name': 'Romaguera-Crona'},
  'email': 'Sincere@april.biz',
  'id': 1,
  'name': 'Leanne Graham',
  'phone': '1-770-736-8031 x56442',
  'username': 'Bret',
  'website': 'hildegard.org'}]

In [44]:
try:
  usuarios = consultar_usuarios()
  lista = []
  for usuario in usuarios:
    lista.append(f'Nombre: {usuario["name"]}, Id: {usuario["id"]}')
  pprint(lista)
except Exception as e:
  print(e)

['Nombre: Leanne Graham, Id: 1',
 'Nombre: Ervin Howell, Id: 2',
 'Nombre: Clementine Bauch, Id: 3',
 'Nombre: Patricia Lebsack, Id: 4',
 'Nombre: Chelsey Dietrich, Id: 5',
 'Nombre: Mrs. Dennis Schulist, Id: 6',
 'Nombre: Kurtis Weissnat, Id: 7',
 'Nombre: Nicholas Runolfsdottir V, Id: 8',
 'Nombre: Glenna Reichert, Id: 9',
 'Nombre: Clementina DuBuque, Id: 10']


In [46]:
def consultar_posts():
    response = requests.get('https://jsonplaceholder.typicode.com/posts')
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception('No se pudieron consultar los posts')
        return []

In [47]:
consultar_posts()[:2]

[{'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto',
  'id': 1,
  'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit',
  'userId': 1},
 {'body': 'est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla',
  'id': 2,
  'title': 'qui est esse',
  'userId': 1}]

In [49]:
def convertir_posts(posts):
    items = []
    for p in posts:
        items.append({
            'id': p['id'],
            'titulo': p['title']
        })
    return items

In [51]:
convertir_posts(consultar_posts())[:2]

[{'id': 1,
  'titulo': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit'},
 {'id': 2, 'titulo': 'qui est esse'}]

Alternativa Lambda Function

In [61]:
def f(post):
  return {
      'id': post['id'],
      'titulo': post['title']
  }

In [65]:
posts = consultar_posts()
tmp = list(map(f, posts))
tmp[:2]

[{'id': 1,
  'titulo': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit'},
 {'id': 2, 'titulo': 'qui est esse'}]

In [64]:
posts = consultar_posts()
tmp = list(map(lambda p: {'id': p['id'], 'titulo': p['title']}, posts))
tmp[:2]

[{'id': 1,
  'titulo': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit'},
 {'id': 2, 'titulo': 'qui est esse'}]

In [66]:
class UsuarioNoEncontrado(Exception):
    def __init__(self, userId):
        self.userId = userId
    def __str__(self):
        return f"El usuario: {self.userId} no ha sido encontrado"

In [68]:
def consultar_usuario():
    try:
        userId = int(input('Ingrese el id del usuario que desea buscar: '))
        usuarios = consultar_usuarios()
        usuario = None
        #buscar el usuario
        for u in usuarios:
            if u['id'] == userId:
                usuario = u
                break
        
        if not usuario:
            raise UsuarioNoEncontrado(userId)
        
        print(f'Nombre: {usuario["name"]}')
        print(f'Username: {usuario["username"]}')
        print(f'Correo: {usuario["email"]}')
        
    except ValueError:
        print('Ingreso un id no valido')
        return None
    except UsuarioNoEncontrado as e:
        print(e)
        return None
    else:
        return usuario


In [72]:
u = consultar_usuario()

Ingrese el id del usuario que desea buscar: 5
Nombre: Chelsey Dietrich
Username: Kamren
Correo: Lucio_Hettinger@annie.ca


In [74]:
def encontrar_posts():
    try:
        usuario = consultar_usuario()
        if not usuario:
            raise UsuarioNoEncontrado()
        posts = consultar_posts()

        filtrados = list(filter(lambda p:p['userId'] == usuario['id'], posts))
        
        copia = usuario.copy()
        copia['posts'] = filtrados
        return copia
    except UsuarioNoEncontrado as e:
        print(e)
        return None

In [75]:
user = encontrar_posts()

Ingrese el id del usuario que desea buscar: 3
Nombre: Clementine Bauch
Username: Samantha
Correo: Nathan@yesenia.net


In [76]:
user

{'address': {'city': 'McKenziehaven',
  'geo': {'lat': '-68.6102', 'lng': '-47.0653'},
  'street': 'Douglas Extension',
  'suite': 'Suite 847',
  'zipcode': '59590-4157'},
 'company': {'bs': 'e-enable strategic applications',
  'catchPhrase': 'Face to face bifurcated interface',
  'name': 'Romaguera-Jacobson'},
 'email': 'Nathan@yesenia.net',
 'id': 3,
 'name': 'Clementine Bauch',
 'phone': '1-463-123-4447',
 'posts': [{'body': 'repellat aliquid praesentium dolorem quo\nsed totam minus non itaque\nnihil labore molestiae sunt dolor eveniet hic recusandae veniam\ntempora et tenetur expedita sunt',
   'id': 21,
   'title': 'asperiores ea ipsam voluptatibus modi minima quia sint',
   'userId': 3},
  {'body': 'eos qui et ipsum ipsam suscipit aut\nsed omnis non odio\nexpedita earum mollitia molestiae aut atque rem suscipit\nnam impedit esse',
   'id': 22,
   'title': 'dolor sint quo a velit explicabo quia nam',
   'userId': 3},
  {'body': 'veritatis unde neque eligendi\nquae quod architecto 

In [84]:
with open('json/usuarios.json', 'r') as f:
  respuestas = json.load(f)

FileNotFoundError: ignored

In [94]:
def guardar_usuario():
    data = encontrar_posts()
    try:
      with open('json/usuarios.json', 'r') as f:
        respuestas = json.load(f)
    except json.decoder.JSONDecodeError:
      respuestas = []
    except FileNotFoundError:
      respuestas = []
    
    respuestas.append(data)

    with open('json/usuarios.json', 'w') as f:
      f.write(json.dumps(respuestas, indent=2))

In [93]:
guardar_usuario()

Ingrese el id del usuario que desea buscar: 1
Nombre: Leanne Graham
Username: Bret
Correo: Sincere@april.biz


In [99]:
def consultar_comentarios_post(postId):
    response = requests.get(f'https://jsonplaceholder.typicode.com/posts/{postId}/comments')
    if response.status_code != 200:
        raise Exception(f'No se pudo consultar los comentarios del post: {postId}')
    comments = response.json()
    return comments
    

In [102]:
def consultar_comentarios():
    try:
        postId = int(input('Ingrese el id del post que desea consultar'))
        posts = consultar_posts()
        post = None
        for p in posts:
            if p['id'] == postId:
                post = p
                break
        if not post:
            raise Exception('Post no encontrado')
        comentarios = consultar_comentarios_post(post['id'])
        return comentarios
    except ValueError:
        print('El id del post es invalido')
        return None
    except Exception as e:
        print(e)
        return None
    


In [104]:
consultar_comentarios()

Ingrese el id del post que desea consultar5656
Post no encontrado


## Excepciones

1. ArithmeticError
    * ZeroDivisionError
2. TypeError
3. ValueError
4. LookupError
    * IndexError
    * KeyError
5. NameError
6. Import Error
    * ModuleNotFoundError
7. OSError
    * FileNotFoundError
    * IsADirectoryError

[Documentación Oficial Python Excepciones](https://docs.python.org/es/3/library/exceptions.html)

In [105]:
try:
    lista.append(5)
except NameError:
    print('Variable no definida')

In [111]:
try:
  with open('juan.txt', 'r') as f:
      print('juan')
except OSError as e:
  print(e)

[Errno 2] No such file or directory: 'juan.txt'


In [110]:
try:
  with open('base', 'r') as f:
      print('acceder a base')
except OSError as e:
  print(e)

[Errno 21] Is a directory: 'base'


In [ ]:
class MiExcepcion(Exception):
    def __init__(self, mensaje, informacion):
        self.mensaje = mensaje
        self.informacion = informacion
    
try:
    raise MiExcepcion("Mi Mensaje",2)
except MiExcepcion as e:
    print(e.mensaje)
    print(e.informacion)

Mi Mensaje
2
